In [1]:
from itertools import chain
from nltk.tokenize import sent_tokenize
from threading import Thread, Lock
from queue import Queue
import time

from htools import *
from jabberwocky.core import GuiTextChunker
from jabberwocky.speech import Speaker

In [2]:
SPEAKER = Speaker()
CHUNKER = GuiTextChunker(max_chars=70)

In [20]:
text = 'Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you prefer. What is that?'
text_simple = 'A dog is sad. He is alone.'

In [21]:
def stream(text):
    for word in text.split(' '):
        yield word + ' '

In [22]:
def read_response(response, errors=None, hide_on_exit=True):
    print('\nresponse:', response)
    try:
        for sent in sent_tokenize(response):
            for chunk in sent.split('\n\n'):
                SPEAKER.speak(chunk)
                if errors:
                    raise errors[0]
    except RuntimeError:
        pass

In [26]:
def old_concurrent_speaking_typing(streamable, conv_mode=False, pause=.18):
    # Stream function provides "typing" effect.
    threads = []
    errors = []
    full_text = ''
    curr_text = ''
    for chunk in stream(streamable):
        full_text += chunk
        curr_text += chunk
        chunked = CHUNKER.add('response', full_text)
        print(chunked, end='')
        if any(char in chunk for char in ('.', '!', '?', '\n\n')):
            if not errors:
                thread = Thread(target=read_response,
                                args=(curr_text, errors, False))
                thread.start()
                threads.append(thread)
            # Make sure this isn't reset until AFTER the speaker thread starts.
            curr_text = ''
        time.sleep(pause)
    if curr_text and not errors:
        read_response(curr_text)
    for thread in threads: thread.join()

In [27]:
old_concurrent_speaking_typing(text)

'response'
Who 
Who are 
Who are you? 
response: Who are you? 

Who are you? I 
Who are you? I am 
Who are you? I am Mr. 
response: I am Mr. 

Who are you? I am Mr. Nichols. 
response: Nichols. 

Who are you? I am Mr. Nichols. You 
Who are you? I am Mr. Nichols. You can 
Who are you? I am Mr. Nichols. You can call 
Who are you? I am Mr. Nichols. You can call me 
Who are you? I am Mr. Nichols. You can call me John. 
response: You can call me John. 

Who are you? I am Mr. Nichols. You can call me John. Or 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., 
response: Or J.T., 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. 
response: if you prefer. 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What 

In [37]:
@coroutine
def read_response_coro():
    # Must send None as an extra last item so that this coroutine knows when 
    # we're done sending in new tokens so it can check for any unread text.
    text = ''
    while True:
        token = yield
        if token is None:
            SPEAKER.speak(sents[0])
        else:
            text += token
            sents = sent_tokenize(text)
            if len(sents) > 1:
                SPEAKER.speak(sents[0])
                text = text.replace(sents[0], '', 1)

In [87]:
# class CoroutinableThread(Thread):
    
#     def __init__(self, target, args=(), kwargs=None):
#         super().__init__(target=target, args=args, kwargs=kwargs)
#         self.target = target(*args, **(kwargs or {}))
        
#     def send(self, val):
#         self.target.send(val)

class CoroutinableThread(Thread):
    
    def __init__(self, target, queue, args=(), kwargs=None):
        super().__init__(target=target, args=args, kwargs=kwargs)
        self.target = target(*args, **(kwargs or {}))
        self.queue = queue
        
    def run(self):
        while True:
            val = self.queue.get()
            self.target.send(val)
            # Must do this after send so our coroutine gets the sentinel.
            if val is None: return

In [88]:
q = Queue()
thread = CoroutinableThread(target=read_response_coro, queue=q, args=())
thread.start()

for t in chain(stream(text_simple), [None]):
    thread.queue.put(t)
    print('LOOP:', t)
    time.sleep(.2)
    
thread.join()

LOOP: A 
LOOP: dog 
LOOP: is 
LOOP: sad. 
LOOP: He 
LOOP: is 
LOOP: alone. 
LOOP: None


In [41]:
# thread = CoroutinableThread(target=read_response_coro, args=())
# thread.start()

# for t in chain(stream(text_simple), [None]):
#     thread.send(t)
#     print(t)
#     time.sleep(.2)

# thread.join()

In [89]:
# def concurrent_speaking_typing(streamable, conv_mode=False, pause=.18):
#     # Stream function provides "typing" effect.
#     threads = []
#     full_text = ''
#     thread = CoroutinableThread(target=read_response_coro, args=())
#     thread.start()
#     for chunk in chain(stream(streamable), [None]):
#         if chunk is not None:
#             full_text += chunk
#             chunked = CHUNKER.add('response', full_text)
#             print(chunked)
#         thread.send(chunk)
#         time.sleep(pause)
#     thread.join()

def concurrent_speaking_typing(streamable, conv_mode=False, pause=.18):
    # Stream function provides "typing" effect.
    full_text = ''
    q = Queue()
    thread = CoroutinableThread(target=read_response_coro, queue=q, args=())
    thread.start()
    for chunk in stream(streamable):
        if conv_mode:
            chunked = CHUNKER.add(
                'conv_transcribed',
                CONV_MANAGER.full_conversation(include_summary=False)
            )
        else:
            chunked = CHUNKER.add('response', full_text)
            full_text += chunk
            print(chunked)
        thread.queue.put(chunk)
        time.sleep(pause)
    thread.queue.put(None)
    thread.join()
#     hide_item(data['interrupt_id'])
#     hide_item(data['query_msg_id'])

In [90]:
concurrent_speaking_typing(text)


Who 

Who are 

Who are you? 

Who are you? I 

Who are you? I am 

Who are you? I am Mr. 

Who are you? I am Mr. Nichols. 

Who are you? I am Mr. Nichols. You 

Who are you? I am Mr. Nichols. You can 

Who are you? I am Mr. Nichols. You can call 

Who are you? I am Mr. Nichols. You can call me 

Who are you? I am Mr. Nichols. You can call me John. 

Who are you? I am Mr. Nichols. You can call me John. Or 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What is 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What is that? 


## Threading scratch

Question: can we start a thread within a thread?

In [9]:
from datetime import datetime as dt

from htools import *

In [20]:
def foo():
    start = time.time()
    while True:
        print(dt.now().strftime('%Y-%m-%d %H:%M:%S'))
        time.sleep(1)
        if time.time() - start >= 5: break

In [21]:
def targ():
    thread = Thread(target=foo, args=())
    thread.start()
    res = [i for i in sleepy_range(10, wait=.5)]
    thread.join()
    return res

In [22]:
res = targ()

2021-08-18 20:53:43
2021-08-18 20:53:44
2021-08-18 20:53:45
2021-08-18 20:53:46
2021-08-18 20:53:47


In [23]:
res

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [25]:
thread = Thread(target=targ, args=())
thread.start()
main_res = [i for i in sleepy_range(5, wait=.1)]
thread.join()

2021-08-18 20:55:12
2021-08-18 20:55:13
2021-08-18 20:55:14
2021-08-18 20:55:15
2021-08-18 20:55:16


In [26]:
main_res

[0, 1, 2, 3, 4]

**Conclusion**

Looks like yes.

## GPT-neo troubleshooting

In [27]:
from jabberwocky.openai_utils import query_gpt_neo

In [36]:
# ~20s first time, <5s second time
res = query_gpt_neo('Hello Mr. President')
print(res[1])

!" "What're you doing here?" "Well, I..." "I haven't been anywhere, sir, and I just wanted one more look." " What's in the case, soldier?" " Oh, one book with a note." "I've just opened it up." "It's a letter from my daughter." "I was just hoping that with your experience in the Orient, you would be able to tell me whether or not she wrote it." "Now, wait." "Now wait, I'll show you it, sir." "Here, I don't want to take any chances." "She wrote the book herself." "All right, come on then, let's get that case out of here." "What's more, soldier, I'll tell you what's in the case just the same." "But if you're lying to me, then there's something very wrong, and I'm gonna get hold of that soldier immediately." "All right." "All right, Mr. President, I'll do it." "You go tell them how good I am." "Yes, sir." "All right, come on out, come on out, come on out, come on out!" "Come on out!" "


In [38]:
# >1 min first time, <1s second time
res = query_gpt_neo('Hello Mr. President', size='1.3B')
print(res[1])

,

The United Nations, with the support the International Criminal Court (ICC) has of course, pursued its quest against alleged war criminals and criminals throughout the world; but the U.S. also has played a leading role in this. The United Nations, on its own motion, has put together a select commission for a fact-finding report to be made, and in its report the United Nations has found over thirty individuals, including at least twenty-five Americans, guilty. Twenty-four, in my personal opinion, are among the most culpable among the guilty. They are: William V. Ayres, a U.S. army staff Sergeant in the 3d Infantry Division in World War II, who had already received the Medal for bravery, a Purple Heart, and a Silver Star for his gallantry on December 8, 1945; James Thomas Bailey, Jr., son of John Lee Bailey, a Marine Corps General in the 2d Infantry Division, who had already received a Silver Star for gallantry for saving the life of a Marine officer in the Battle of Tarawa, on Decemb

In [35]:
# ~17s first time, <1s second time
res = query_gpt_neo('Hello Mr. President', size='125M')
print(res[1])

, I have a question. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United States is doing its job. I have a question about the way in which the United Sta

## Free GPT-J api

In [63]:
import requests

from jabberwocky.openai_utils import query_gpt3, MockFunctionException

In [41]:
# Copy-pasted from github example.
context = "In a shocking finding, scientist discovered a herd of unicorns "\
"living in a remote, previously unexplored valley, in the Andes Mountains. "\
"Even more surprising to the researchers was the fact that the unicorns "\
"spoke perfect English."
payload = {
    "context": context,
    "token_max_length": 512,
    "temperature": 1.0,
    "top_p": 0.9,
}
response = requests.post("http://api.vicgalle.net:5000/generate", 
                         params=payload).json()
print(response)

{'model': 'GPT-J-6B', 'compute_time': 8.96641230583191, 'text': '\n\nThere were actually 9 unicorns in the herd, and were eventually killed and skinned by the farmer, for their excellent hides, according to a study published on Monday.\n\nFor years, scientists had believed that the bovine population of South America was still in its infant stage of evolution. While they knew there was something special about the animals, they still could not figure out what it was.\n\nOnly when the bovine population became extinct, something which was recorded in the Book of Genesis, were the scientists able to finally figure out that these enormous, heavy-browed cattle were a new species of unicorn.\n\nScientists did find a way to preserve the history of these unicorns, when they noticed the family of men responsible for killing them, began speaking in perfect English.\n\nThrough a series of interviews, they found out that a man named Pedro Ferro had spent his life gathering "fat unicorns", although h

In [45]:
response['prompt']

'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.'

In [43]:
print(response['text'])



There were actually 9 unicorns in the herd, and were eventually killed and skinned by the farmer, for their excellent hides, according to a study published on Monday.

For years, scientists had believed that the bovine population of South America was still in its infant stage of evolution. While they knew there was something special about the animals, they still could not figure out what it was.

Only when the bovine population became extinct, something which was recorded in the Book of Genesis, were the scientists able to finally figure out that these enormous, heavy-browed cattle were a new species of unicorn.

Scientists did find a way to preserve the history of these unicorns, when they noticed the family of men responsible for killing them, began speaking in perfect English.

Through a series of interviews, they found out that a man named Pedro Ferro had spent his life gathering "fat unicorns", although he did not know what he was doing when he started his hunt.

The conditions 

In [47]:
# Streaming mode not supported (no error, but we won't get tokens streamed to
# us).
payload = {
    "context": context,
    "token_max_length": 100,
    "temperature": 1.0,
    "top_p": 0.9,
    "stream": True
}
response = requests.post("http://api.vicgalle.net:5000/generate", 
                         params=payload).json()
print(response)

{'model': 'GPT-J-6B', 'compute_time': 1.8529937267303467, 'text': ' They were also able to teleport to the lab where the study was being conducted and learn every word of the complex experiment.\n\nWhile looking into the study on unicorns, I found this interesting article about a study done to explain the prevalence of divination in various cultures and religions. There are those who call themselves “Herodian” or “Theonomist” but most of them make no such claim. They just believe they are following the will of God, and that', 'prompt': 'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.', 'token_max_length': 100, 'temperature': 1.0, 'top_p': 0.9, 'stop_sequence': None}


In [55]:
# Stop sequence works slightly differently here.
# 1. Use "stop_sequence" param, not "stop" like in openai api.
# 2. Completion will include the stop sequence if it was generated. We should
# strip it in postprocessing.
payload = {
    "context": context,
    "token_max_length": 100,
    "temperature": 1.0,
    "top_p": 0.9,
    "stop_sequence": ', '
}
response = requests.post("http://api.vicgalle.net:5000/generate", 
                         params=payload).json()
print(response['text'])



Fascinating video of a group of 200 "Gigantic" sloths walking and climbing along the mountainside in Costa Rica.

Exotic animal giraffes are normal-sized for their species, 


In [56]:
params(query_gpt_neo)

{'prompt': <Parameter "prompt">,
 'top_k': <Parameter "top_k=None">,
 'top_p': <Parameter "top_p=None">,
 'temperature': <Parameter "temperature=1.0">,
 'repetition_penalty': <Parameter "repetition_penalty=None">,
 'max_tokens': <Parameter "max_tokens=250">,
 'api_key': <Parameter "api_key=None">,
 'size': <Parameter "size: ('125M', '1.3B', '2.7B') = '2.7B'">,
 'kwargs': <Parameter "**kwargs">}

In [59]:
params(query_gpt3)

{'prompt': <Parameter "prompt">,
 'engine_i': <Parameter "engine_i=0">,
 'temperature': <Parameter "temperature=0.7">,
 'frequency_penalty': <Parameter "frequency_penalty=0.0">,
 'max_tokens': <Parameter "max_tokens=50">,
 'logprobs': <Parameter "logprobs=None">,
 'stream': <Parameter "stream=False">,
 'mock': <Parameter "mock=False">,
 'return_full': <Parameter "return_full=False">,
 'strip_output': <Parameter "strip_output=True">,
 'mock_func': <Parameter "mock_func=None">,
 'mock_mode': <Parameter "mock_mode: ('raise', 'warn', 'ignore') = 'raise'">,
 'kwargs': <Parameter "**kwargs">}

In [81]:
def query_gpt_j(prompt, temperature=0.7, max_tokens=50, **kwargs):
    params = {'context': prompt,
              'token_max_length': max_tokens,
              'temperature': temperature,
              'top_p': kwargs.pop('top_p', 1.0)}
    
    # Ensure that we end up with a list AND that stop is still Falsy if user 
    # explicitly passes in stop=None.
    stop = tolist(kwargs.pop('stop', None) or [])
    if stop: params['stop_sequence'] = stop[0]
        
    # Must keep this after the block of stop-related logic above.
    if kwargs: warnings.warn('GPT-J api does not support other kwargs.')
    
    try:
        res = requests.post('http://api.vicgalle.net:5000/generate', 
                            params=params)
        res.raise_for_status()
    except Exception as e:
        raise MockFunctionException(str(e)) from None
    res = res.json()
        
    # Endpoint doesn't support multiple stop sequences so we have to 
    # postprocess. Even with a single stop sequence, it includes it while gpt3
    # and my gpt-neo function exclude it, so we need to handle that here.
    idx = min([i for i in map(res['text'].find, stop) if i >= 0] or [None])
    completion = res['text'][:idx]
    return res['prompt'], completion

In [82]:
prompt = """This is a conversation with Barack Obama.

Me: Hello Barack.

Barack Obama:"""

query_gpt_j(prompt, stop='Me: ')

('This is a conversation with Barack Obama.\n\nMe: Hello Barack.\n\nBarack Obama:',
 ' Hi.\n\n')

In [86]:
query_gpt_j(prompt, stop='Me: ')

('This is a conversation with Barack Obama.\n\nMe: Hello Barack.\n\nBarack Obama:',
 ' Hello.\n\n')